In [1]:
import pennylane as qml
from pennylane import numpy as np
from measurement_memory import evaluate_eigenstate, evaluate_eigenstate_MM, GradientDescent

## Ising Hamiltonian

In [2]:
# Problem Hamiltonian
qubits = 15
coeffs = [1.0]*(qubits*(qubits-1)//2 + qubits)
obs = [qml.PauliZ(i)@qml.PauliZ(j) for i in range(qubits) for j in range(i, qubits)]
H = qml.Hamiltonian(coeffs, obs, grouping_type='qwc')

In [3]:
p = 1 
def ansatz(params, qubits=qubits, depth=p):
    for q in range(qubits):
            qml.RY(params[q], wires=q)
    for d in range(1,depth+1):
        for q in range(qubits-1):
            qml.CNOT(wires=[q,q+1])
        for q in range(qubits):
            qml.RY(params[d*qubits+q], wires=q)

In [4]:
dev = qml.device("default.qubit", wires=qubits, shots=9999)

@qml.qnode(dev)
def sample_circuit(params):
    ansatz(params)
    return qml.counts()

In [9]:
def cost(x):
    return evaluate_eigenstate_MM(sample_circuit(x), H, memory=M, memory_states=5000)

In [5]:
init_params = np.random.rand((p+1)*qubits)
max_itr = 100
gradient_method = 'parameter_shift'

In [10]:
%%time
M = {}
params = init_params
print("itr {}, E = {}".format(0, cost(params)))
for itr in range(max_itr):
    params = GradientDescent(cost, params, gradient_method, learning_rate=0.01)
    obj_value = cost(params)
    print("itr {}, E = {}".format(itr+1, obj_value))

itr 0, E = 23.96679667966797
itr 1, E = 19.765176517651764
itr 2, E = 15.996999699969997
itr 3, E = 13.832583258325833
itr 4, E = 11.955595559555956
itr 5, E = 10.34103410341034
itr 6, E = 9.355935593559355
itr 7, E = 8.117611761176118
itr 8, E = 7.401740174017402
itr 9, E = 6.74967496749675
itr 10, E = 6.262026202620262
itr 11, E = 5.222722272227223
itr 12, E = 4.750875087508751
itr 13, E = 3.9877987798779877
itr 14, E = 3.681168116811681
itr 15, E = 3.5203520352035205
itr 16, E = 2.9574957495749574
itr 17, E = 2.415241524152415
itr 18, E = 2.004600460046005
itr 19, E = 1.688968896889689
itr 20, E = 1.2417241724172416
itr 21, E = 1.1399139913991398
itr 22, E = 0.9320932093209321
itr 23, E = 0.40344034403440343
itr 24, E = 0.24742474247424742
itr 25, E = 0.016401640164016403
itr 26, E = 0.17421742174217422
itr 27, E = -0.4880488048804881
itr 28, E = -0.46004600460046
itr 29, E = -0.7236723672367237
itr 30, E = -0.9418941894189419
itr 31, E = -1.009100910091009
itr 32, E = -1.2603260326

#### No MM

In [6]:
def cost0(x):
    return evaluate_eigenstate(sample_circuit(x), H)

In [7]:
%%time
params = init_params
print("itr {}, E = {}".format(0, cost0(params)))
for itr in range(max_itr):
    params = GradientDescent(cost0, params, gradient_method, learning_rate=0.01)
    obj_value = cost0(params)
    print("itr {}, E = {}".format(itr+1, obj_value))

itr 0, E = 23.78017801780178
itr 1, E = 19.77097709770977
itr 2, E = 16.376837683768375
itr 3, E = 14.156215621562156
itr 4, E = 12.077607760776077
itr 5, E = 10.495649564956496
itr 6, E = 9.54895489548955
itr 7, E = 8.895089508950894
itr 8, E = 7.508950895089509
itr 9, E = 6.427842784278428
itr 10, E = 5.981998199819982
itr 11, E = 4.95029502950295
itr 12, E = 4.561256125612561
itr 13, E = 4.168816881688169
itr 14, E = 3.5173517351735173
itr 15, E = 3.0805080508050806
itr 16, E = 3.0035003500350035
itr 17, E = 2.4804480448044806
itr 18, E = 2.127812781278128
itr 19, E = 1.8871887188718872
itr 20, E = 1.3281328132813282
itr 21, E = 1.1443144314431444
itr 22, E = 0.8690869086908691
itr 23, E = 0.6724672467246725
itr 24, E = 0.41924192419241924
itr 25, E = 0.1348134813481348
itr 26, E = -0.19641964196419642
itr 27, E = -0.49504950495049505
itr 28, E = -0.49904990499049906
itr 29, E = -0.6302630263026303
itr 30, E = -0.8974897489748975
itr 31, E = -1.1073107310731074
itr 32, E = -1.321532

In [16]:
@qml.qnode(dev, diff_method="parameter-shift")
def exp_circuit(params):
    ansatz(params)
    return qml.expval(H)

In [17]:
def cost1(x):
    return exp_circuit(x)

In [18]:
%%time
opt = qml.GradientDescentOptimizer(stepsize=0.01)

params = np.array(init_params, requires_grad=True)
print("itr {}, E = {}".format(0, cost1(params)))
for itr in range(max_itr):
    params = opt.step(cost1, params)
    obj_value = cost1(params)
    print("itr {}, E = {}".format(itr+1, obj_value))

itr 0, E = 27.723472347234726
itr 1, E = 21.994099409940993
itr 2, E = 17.64366436643664
itr 3, E = 14.567356735673567
itr 4, E = 12.343134313431342
itr 5, E = 10.383738373837383
itr 6, E = 9.064006400640066
itr 7, E = 8.155115511551156
itr 8, E = 7.031403140314032
itr 9, E = 6.602560256025604
itr 10, E = 5.821682168216821
itr 11, E = 5.513251325132513
itr 12, E = 4.4003400340034
itr 13, E = 3.609060906090609
itr 14, E = 3.3142314231423136
itr 15, E = 3.081808180818081
itr 16, E = 2.41134113411341
itr 17, E = 2.0809080908090793
itr 18, E = 1.7108710871087105
itr 19, E = 1.6360636063606364
itr 20, E = 1.1104110411041106
itr 21, E = 0.7375737573757375
itr 22, E = 0.5263526352635257
itr 23, E = 0.41074107410741
itr 24, E = 0.21552155215521507
itr 25, E = -0.21052105210521044
itr 26, E = -0.3097309730973103
itr 27, E = -0.5105510551055106
itr 28, E = -0.6737673767376753
itr 29, E = -0.7633763376337619
itr 30, E = -1.142614261426144
itr 31, E = -1.063406340634062
itr 32, E = -1.352235223522

## Molecular Hamiltonian

In [11]:
import pennylane as qml
from pennylane import numpy as np
from measurement_memory import evaluate_eigenstate, evaluate_eigenstate_MM, GradientDescent

### LiH

In [12]:
from measurement_memory import str_to_Pauli
qubits = 4
Hg=[]

path = 'LiH_Hamiltonian.txt'
with open(path) as f:
    group_ops = []
    group_coeffs = []
    for line in f.readlines():
        s = line.split(' ')
        if s[0] == '\n' or s[0] == None:
            Hg.append(qml.Hamiltonian(group_coeffs, str_to_Pauli(group_ops), grouping_type='qwc'))
            group_ops = []
            group_coeffs = []
        else:
            group_ops.append(s[0])
            if s[1][-2:] == '\n':
                group_coeffs.append(float(s[1][:-2]))
            else:
                group_coeffs.append(float(s[1]))

In [13]:
print("Number of qubits: {:}".format(qubits))
big_groups = []
for i,h in enumerate(Hg):
    if len(h.ops) >= 10:
        big_groups.append(i)
    print("Group {}".format(i+1))
    print(h)
    
print("\n Big Groups : ",big_groups)

Number of qubits: 4
Group 1
  (-0.364746) [Z3]
+ (-0.096022) [Z0]
+ (0.096022) [Z2]
+ (0.364746) [Z1]
+ (-0.206128) [Z0 Z1]
+ (-0.206128) [Z2 Z3]
+ (-0.145438) [Z0 Z2]
+ (-0.095216) [Z1 Z3]
+ (0.110811) [Z0 Z3]
+ (0.110811) [Z1 Z2]
+ (-0.063673) [Z1 Z2 Z3]
+ (-0.05604) [Z0 Z1 Z2]
+ (0.05604) [Z0 Z2 Z3]
+ (0.063673) [Z0 Z1 Z3]
+ (0.080334) [Z0 Z1 Z2 Z3]
Group 2
  (0.012585) [X0]
+ (0.012585) [X2]
+ (-0.012585) [X0 Z1]
+ (-0.007265) [X0 Z3]
+ (0.002667) [X0 X2]
+ (0.007265) [Z1 X2]
+ (0.012585) [X2 Z3]
+ (-0.002667) [X0 Z1 X2]
+ (0.002667) [X0 X2 Z3]
+ (0.007265) [X0 Z1 Z3]
+ (0.007265) [Z1 X2 Z3]
+ (-0.002667) [X0 Z1 X2 Z3]
Group 3
  (0.002792) [X1]
+ (0.002792) [X3]
+ (-0.02964) [X0 X1]
+ (-0.02964) [X2 X3]
+ (-0.001271) [X0 X3]
+ (-0.001271) [X1 X2]
+ (0.009327) [X1 X3]
+ (-0.008195) [X0 X2 X3]
+ (-0.008195) [X0 X1 X2]
+ (0.007499) [X0 X1 X3]
+ (0.007499) [X1 X2 X3]
+ (0.028926) [X0 X1 X2 X3]
Group 4
  (0.02964) [Y0 Y1]
+ (0.02964) [Y2 Y3]
+ (0.028926) [Y0 Y1 Y2 Y3]
Group 5
  (-0.0167

In [14]:
from measurement_memory import get_measurement_list, measurement_rotation

Measurement_list = get_measurement_list(Hg, qubits)
print(Measurement_list)

[['PauliZ', 'PauliZ', 'PauliZ', 'PauliZ'], ['PauliX', 'PauliZ', 'PauliX', 'PauliZ'], ['PauliX', 'PauliX', 'PauliX', 'PauliX'], ['PauliY', 'PauliY', 'PauliY', 'PauliY'], ['PauliZ', 'PauliX', 'PauliZ', 'PauliX'], ['PauliZ', 'PauliZ', 'PauliX', 'PauliZ'], ['PauliZ', 'PauliZ', 'PauliX', 'PauliX'], ['PauliZ', 'PauliZ', 'PauliY', 'PauliY'], ['PauliX', 'PauliZ', 'PauliZ', 'PauliZ'], ['PauliX', 'PauliZ', 'PauliX', 'PauliX'], ['PauliX', 'PauliZ', 'PauliY', 'PauliY'], ['PauliX', 'PauliZ', 'PauliZ', 'PauliX'], ['PauliX', 'PauliX', 'PauliZ', 'PauliZ'], ['PauliY', 'PauliY', 'PauliZ', 'PauliZ'], ['PauliX', 'PauliX', 'PauliX', 'PauliZ'], ['PauliY', 'PauliY', 'PauliX', 'PauliZ'], ['PauliX', 'PauliX', 'PauliY', 'PauliY'], ['PauliY', 'PauliY', 'PauliX', 'PauliX'], ['PauliX', 'PauliX', 'PauliZ', 'PauliX'], ['PauliY', 'PauliY', 'PauliZ', 'PauliX'], ['PauliZ', 'PauliZ', 'PauliZ', 'PauliX'], ['PauliZ', 'PauliX', 'PauliX', 'PauliZ'], ['PauliZ', 'PauliX', 'PauliX', 'PauliX'], ['PauliZ', 'PauliX', 'PauliY', 'P

In [15]:
p = 4
def ansatz(params, qubits=qubits, depth=p):
    for q in range(qubits):
        qml.U3(params[3*q],params[3*q+1],params[3*q+2], wires=q)
        #print(3*q, 3*q+1, 3*q+2)
    for d in range(1,depth+1):
        #print(d)
        for q in range(qubits-1):
            qml.CNOT(wires=[q,q+1])
        for q in range(qubits):
            qml.U3(params[3*(d*qubits+q)], params[3*(d*qubits+q)+1], params[3*(d*qubits+q)+2], wires=q)
        #    print(3*(d*qubits+q), 3*(d*qubits+q)+1, 3*(d*qubits+q)+2)

In [16]:
dev = qml.device("default.qubit", wires=qubits, shots=9999)

@qml.qnode(dev)
def sample_circuit(params, obs):
    ansatz(params)
    measurement_rotation(obs)
    return qml.counts()

In [17]:
def cost(x):
    E = 0
    for i,h in enumerate(Hg):
        if i in big_groups:
            E += evaluate_eigenstate_MM(sample_circuit(x, Measurement_list[i]), h, memory=M[i], memory_states=5000)
        else:
            E += evaluate_eigenstate(sample_circuit(x, Measurement_list[i]), h)
    return E

In [18]:
init_params = np.random.rand(3*(p+1)*qubits)
max_itr = 100
gradient_method = 'parameter_shift'

In [ ]:
%%time

M = [{} for _ in range(len(Hg))]
params = init_params
print("itr {}, E = {}".format(0, cost(params)))
for itr in range(max_itr):
    params = GradientDescent(cost, params, gradient_method, learning_rate=4/(itr+1))
    obj_value = cost(params)
    print("itr {}, E = {}".format(itr+1, obj_value))

itr 0, E = -0.3026822084208422
itr 1, E = -0.2720609546954696
itr 2, E = -0.4002103978397841
itr 3, E = -0.33641099109911
itr 4, E = -0.5463766098609865
itr 5, E = -0.6050242634263426
itr 6, E = -0.6611881828182817
itr 7, E = -0.6953330901090109
itr 8, E = -0.7064957729772976
itr 9, E = -0.7166447084708472
itr 10, E = -0.7194209878987899
itr 11, E = -0.7231116375637564
itr 12, E = -0.7268394663466345
itr 13, E = -0.7269185756575656
itr 14, E = -0.7298440530053008
itr 15, E = -0.7316570609060908
itr 16, E = -0.7337733793379339
itr 17, E = -0.7321400124012398
itr 18, E = -0.7334144654465445
itr 19, E = -0.7392579513951392
itr 20, E = -0.7360118569856988
itr 21, E = -0.7389640902090208
itr 22, E = -0.7398841022102212
itr 23, E = -0.7396583236323634
itr 24, E = -0.7391549900990099
itr 25, E = -0.7451771701170117
itr 26, E = -0.7387806276627663
itr 27, E = -0.7435344064406443
itr 28, E = -0.7405697067706768
itr 29, E = -0.742946717471747
itr 30, E = -0.7410549082908291
itr 31, E = -0.739792

In [ ]:
M

#### No MM

In [35]:
def cost0(x):
    E = 0
    for i,h in enumerate(Hg):
        E += evaluate_eigenstate(sample_circuit(x, Measurement_list[i]), h)
    return E

In [36]:
%%time
# No MM
params = init_params
print("itr {}, E = {}".format(0, cost0(params)))
for itr in range(max_itr):
    params = GradientDescent(cost0, params, gradient_method, learning_rate=4/(itr+1))
    obj_value = cost0(params)
    print("itr {}, E = {}".format(itr+1, obj_value))

itr 0, E = -0.057858492049204985
itr 1, E = -0.08225262486248626
itr 2, E = -0.09462366136613666
itr 3, E = -0.11052911751175121
itr 4, E = -0.11718907630763085
itr 5, E = -0.14259412281228115
itr 6, E = -0.15693004060406043
itr 7, E = -0.17545294429442948
itr 8, E = -0.19763912571257136
itr 9, E = -0.2157406494649466
itr 10, E = -0.23210430903090315
itr 11, E = -0.25182292309230936
itr 12, E = -0.2669333579357937
itr 13, E = -0.2834854655465548
itr 14, E = -0.3032476617661766
itr 15, E = -0.3153222378237823
itr 16, E = -0.33073073267326725
itr 17, E = -0.3421683558355836
itr 18, E = -0.3558695785578558
itr 19, E = -0.3643709588958898
itr 20, E = -0.37426878867886787
itr 21, E = -0.3788158237823784
itr 22, E = -0.38732672727272766
itr 23, E = -0.39141031723172326
itr 24, E = -0.3970795595559557
itr 25, E = -0.3981818071807182
itr 26, E = -0.40342059905990596
itr 27, E = -0.40866852845284535
itr 28, E = -0.41291928172817305
itr 29, E = -0.4124864084408442
itr 30, E = -0.4191278781878188

### BeH2

In [2]:
from measurement_memory import str_to_Pauli
qubits = 6
Hg=[]

path = 'BeH2_Hamiltonian.txt'
with open(path) as f:
    group_ops = []
    group_coeffs = []
    for line in f.readlines():
        s = line.split(' ')
        if s[0] == '\n' or s[0] == None:
            Hg.append(qml.Hamiltonian(group_coeffs, str_to_Pauli(group_ops), grouping_type='qwc'))
            group_ops = []
            group_coeffs = []
        else:
            group_ops.append(s[0])
            if s[1][-2:] == '\n':
                group_coeffs.append(float(s[1][:-2]))
            else:
                group_coeffs.append(float(s[1]))

In [3]:
print("Number of qubits: {:}".format(qubits))
big_groups = []
for i,h in enumerate(Hg):
    if len(h.ops) >= 10:
        big_groups.append(i)
    print("Group {}".format(i+1))
    print(h)
    
print("\n Big Groups : ",big_groups)

Number of qubits: 6
Group 1
  (-0.325651) [Z2]
+ (-0.325651) [Z5]
+ (-0.143021) [Z0]
+ (-0.143021) [Z3]
+ (0.172191) [Z1]
+ (0.172191) [Z4]
+ (0.038195) [Z1 Z2]
+ (0.038195) [Z4 Z5]
+ (0.104962) [Z0 Z1]
+ (0.104962) [Z3 Z4]
+ (0.116134) [Z0 Z3]
+ (0.123367) [Z0 Z5]
+ (0.123367) [Z2 Z3]
+ (0.133557) [Z2 Z5]
+ (0.136055) [Z0 Z2]
+ (0.136055) [Z3 Z5]
+ (0.094064) [Z0 Z3 Z4]
+ (0.094064) [Z0 Z1 Z3]
+ (0.097795) [Z0 Z1 Z5]
+ (0.097795) [Z2 Z3 Z4]
+ (0.099152) [Z0 Z4 Z5]
+ (0.099152) [Z1 Z2 Z3]
+ (0.105708) [Z1 Z2 Z5]
+ (0.105708) [Z2 Z4 Z5]
+ (0.174763) [Z0 Z1 Z2]
+ (0.174763) [Z3 Z4 Z5]
+ (0.098003) [Z0 Z1 Z3 Z4]
+ (0.102525) [Z0 Z1 Z4 Z5]
+ (0.102525) [Z1 Z2 Z3 Z4]
+ (0.112045) [Z1 Z2 Z4 Z5]
Group 2
  (-0.161019) [X2]
+ (-0.161019) [X5]
+ (-0.05911) [X0]
+ (-0.05911) [X3]
+ (-0.038098) [X0 X2]
+ (-0.038098) [X3 X5]
+ (0.011986) [X0 X5]
+ (0.011986) [X2 X3]
+ (0.013745) [X0 X3]
+ (0.013836) [X2 X5]
+ (0.05911) [X0 Z1]
+ (0.05911) [X3 Z4]
+ (0.161019) [Z1 X2]
+ (0.161019) [Z4 X5]
+ (-0.0138

In [4]:
from measurement_memory import get_measurement_list, measurement_rotation

Measurement_list = get_measurement_list(Hg, qubits)

In [5]:
p = 5
def ansatz(params, qubits=qubits, depth=p):
    for q in range(qubits):
        qml.U3(params[3*q],params[3*q+1],params[3*q+2], wires=q)
    for d in range(1,depth+1):
        for q in range(qubits-1):
            qml.CNOT(wires=[q,q+1])
        for q in range(qubits):
            qml.U3(params[3*(d*qubits+q)], params[3*(d*qubits+q)+1], params[3*(d*qubits+q)+2], wires=q)

In [6]:
dev = qml.device("default.qubit", wires=qubits, shots=9999)

@qml.qnode(dev)
def sample_circuit(params, obs):
    ansatz(params)
    measurement_rotation(obs)
    return qml.counts()

In [7]:
def cost(x):
    E = 0
    for i,h in enumerate(Hg):
        if i in big_groups:
            E += evaluate_eigenstate_MM(sample_circuit(x, Measurement_list[i]), h, memory=M[i], memory_states=5000)
        else:
            E += evaluate_eigenstate(sample_circuit(x, Measurement_list[i]), h)
    return E

In [8]:
init_params = np.random.rand(3*(p+1)*qubits)
max_itr = 100
gradient_method = 'parameter_shift'

In [10]:
%%time

M = [{} for _ in range(len(Hg))]
params = init_params
print("itr {}, E = {}".format(0, cost(params)))
for itr in range(max_itr):
    params = GradientDescent(cost, params, gradient_method, learning_rate=4/(itr+1))
    obj_value = cost(params)
    print("itr {}, E = {}".format(itr+1, obj_value))

itr 0, E = 0.22163859595959587
itr 1, E = 0.15527069896989698
itr 2, E = -0.35651797689768994
itr 3, E = -0.6350654118411841
itr 4, E = -0.8070287359735974
itr 5, E = -0.9153985129512949
itr 6, E = -0.9724304975497553
itr 7, E = -1.0260780425042504
itr 8, E = -1.0735419176917693
itr 9, E = -1.111644238923892
itr 10, E = -1.1295137462746274
itr 11, E = -1.1546632838283826
itr 12, E = -1.1747753982398237
itr 13, E = -1.1936678972897288
itr 14, E = -1.205158743974396
itr 15, E = -1.2254691772177224
itr 16, E = -1.227116285528552
itr 17, E = -1.249437842684268
itr 18, E = -1.2616201079107907
itr 19, E = -1.2652697002700264
itr 20, E = -1.2809624865486557
itr 21, E = -1.2818705681568152
itr 22, E = -1.296547978897889
itr 23, E = -1.2989397958795887
itr 24, E = -1.3082085943594353
itr 25, E = -1.316492249924993
itr 26, E = -1.3126328137813772
itr 27, E = -1.321259800280029
itr 28, E = -1.342614682368237
itr 29, E = -1.3390627609760972
itr 30, E = -1.344530782778278
itr 31, E = -1.34029734883